In [ ]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from timeit import default_timer as timer

from sklearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.metrics import  mean_squared_error, r2_score
import xgboost as xgb

# import functions
from src.modeling.modeling_functions import error_analysis

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '%.2f' % x) # change decimal places

RSEED = 42

## Loading the processed dataframe (sugarbeet and openweather station data - all fields)

In [ ]:
df = pd.read_pickle('pickles/df_openweather_monthly_sugarbeet.pkl')
df

In [ ]:
# convert to string for categorizing
df['seednames_coded'] = df['seednames_coded'].astype(str)
df['pollinator_comp'] = df['pollinator_comp'].astype(str)
df['ms_comp'] = df['ms_comp'].astype(str)
df['otype_comp'] = df['otype_comp'].astype(str)

In [ ]:
df.shape

In [ ]:
# dropping columns 
df.drop(['betaine_nir', 
         'cry_nir', 
         'dm_nir', 
         'invert_nir', 
         'mark_nir', 
         #'sc_nir',
         'csy_nir', 
         'totaln_nir', 
         'obj',  
         'seriesid', 
         'x', 
         'y', 
         'ms_comp',
         'otype_comp', 
         #'pollinator_comp',
         #'seednames_coded',
         'region',
         'station_location'
         ], axis=1, inplace=True)
df.columns

In [ ]:
df.shape

In [ ]:
# reset index after drpping columns
df = df.reset_index(drop=True)

#### Creating the pipeline

In [ ]:
# creating list for categorical predictors/features 
cat_features = list(df.columns[df.dtypes==object])
cat_features

In [ ]:
# creating list for numerical predictors/features
# since 'sc_nir' is our target variable we will exclude this feature from the list of numerical predictors 
# latitude and longitude are also excluded to avoid location influence on prediction
num_features = [
 'dew_point_monthly_10.0',
 'dew_point_monthly_4.0',
 'dew_point_monthly_5.0',
 'dew_point_monthly_6.0',
 'dew_point_monthly_7.0',
 'dew_point_monthly_8.0',
 'dew_point_monthly_9.0',
 'humidity_monthly_10.0',
 'humidity_monthly_4.0',
 'humidity_monthly_5.0',
 'humidity_monthly_6.0',
 'humidity_monthly_7.0',
 'humidity_monthly_8.0',
 'humidity_monthly_9.0',
 'pressure_monthly_10.0',
 'pressure_monthly_4.0',
 'pressure_monthly_5.0',
 'pressure_monthly_6.0',
 'pressure_monthly_7.0',
 'pressure_monthly_8.0',
 'pressure_monthly_9.0',
 'temp_max_monthly_10.0',
 'temp_max_monthly_4.0',
 'temp_max_monthly_5.0',
 'temp_max_monthly_6.0',
 'temp_max_monthly_7.0',
 'temp_max_monthly_8.0',
 'temp_max_monthly_9.0',
 'temp_min_monthly_10.0',
 'temp_min_monthly_4.0',
 'temp_min_monthly_5.0',
 'temp_min_monthly_6.0',
 'temp_min_monthly_7.0',
 'temp_min_monthly_8.0',
 'temp_min_monthly_9.0',
 'temp_monthly_10.0',
 'temp_monthly_4.0',
 'temp_monthly_5.0',
 'temp_monthly_6.0',
 'temp_monthly_7.0',
 'temp_monthly_8.0',
 'temp_monthly_9.0',
 'wind_deg_monthly_10.0',
 'wind_deg_monthly_4.0',
 'wind_deg_monthly_5.0',
 'wind_deg_monthly_6.0',
 'wind_deg_monthly_7.0',
 'wind_deg_monthly_8.0',
 'wind_deg_monthly_9.0',
 'wind_speed_monthly_10.0',
 'wind_speed_monthly_4.0',
 'wind_speed_monthly_5.0',
 'wind_speed_monthly_6.0',
 'wind_speed_monthly_7.0',
 'wind_speed_monthly_8.0',
 'wind_speed_monthly_9.0'
]
num_features

#### Train-Test-Split

In [ ]:
# define predictors and target variable
X = df.drop('sc_nir', axis=1)
y = df['sc_nir']
print(f"We have {X.shape[0]} observations in our dataset and {X.shape[1]} features")
print(f"Our target vector has also {y.shape[0]} values")

In [ ]:
# split into train and test set (train set: 70%, test set: 30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RSEED) 

In [ ]:
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

#### Preprocessing Pipeline

In [ ]:
# Pipeline for categorical features 
cat_pipeline = Pipeline([
    ('1hot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
# Pipeline for numerical features
num_pipeline = Pipeline([
    ('std_scaler', RobustScaler())
])

In [ ]:
# complete pipeline for numerical features
# apply transformers to numerical pipeline
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])

### Predictive modeling using Pipeline and GridSearch

#### XGBoost

In [ ]:
# building a full pipeline with our preprocessor and the RandomForestRegressor
pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('xgb', xgb.XGBRegressor(n_estimators = 100))
])

In [ ]:
# making predictions on the training set using cross validation as well as calculating the probabilities
# cross_val_predict expects an estimator (model), X, y and number of cv-splits (cv)
y_train_predicted = cross_val_predict(pipe, X_train, y_train, cv=5)

In [ ]:
# Mean Squared Error
print('MSE XGBoost Train:\n', mean_squared_error(y_train, y_train_predicted))

# Root Mean Squared Error
print('RMSE XGBoost Train:\n', mean_squared_error(y_train, y_train_predicted, squared = False))

# R^2 Score
print('R^2 XGBoost Train:\n', r2_score(y_train, y_train_predicted))


#### Optimizing via GridSearch

In [ ]:
# defining parameter space for grid-search
# add 'xgb__' infront of the corresponding hyperparameters
param = {'xgb__max_depth': [10,20,30,40,50],
                  'xgb__max_features': ["sqrt"],
                  'xgb__max_leaf_nodes': [500, 5000],
                  'xgb__min_samples_split': [10, 50, 100]}

grid = GridSearchCV(pipe, param_grid=param, scoring = r2_score, cv=5,
                           verbose=1, n_jobs=-1)

In [ ]:
# train the model
grid.fit(X_train, y_train)

In [ ]:
# show best parameters
print('Best score:\n{:.2f}'.format(grid.best_score_))
print("Best parameters:\n{}".format(grid.best_params_))

In [ ]:
# save best model (including fitted preprocessing steps) as best_model 
best_model = grid.best_estimator_
best_model

In [ ]:
# calculating the mean squared error, root mean squared error and r^2 score for the test set with the optimized model
y_test_predicted = best_model.predict(X_test)

# Mean Squared Error
print('MSE XGBoost Test:\n', mean_squared_error(y_test, y_test_predicted))

# Root Mean Squared Error
print('RMSE XGBoost Test:\n', mean_squared_error(y_test, y_test_predicted, squared = False))

# R^2 Score
print('R^2 XGBoost Test:\n', r2_score(y_test, y_test_predicted))

In [ ]:
# evaluate the error
error_analysis(y_test,y_test_predicted)

#### Feature importance

In [ ]:
# fit pipe with train data
model = pipe.fit(X_train, y_train)

In [ ]:
# get coefficients
coefs = model.steps[1][1].feature_importances_

In [ ]:
# get feature names
feature_names = model.steps[0][1].get_feature_names_out()

In [ ]:
# zip coefficients and names together and make a DataFrame
zipped = zip(feature_names, coefs)
df = pd.DataFrame(zipped, columns=["feature", "value"])
# sort the features by the absolute value of their coefficient
df["abs_value"] = df["value"].apply(lambda x: abs(x))
df["colors"] = df["value"].apply(lambda x: "#2f7055" if x > 0 else "red")
df = df.sort_values("abs_value", ascending=False)

In [ ]:
# plot feature importance
fig, ax = plt.subplots(1, 1, figsize=(12, 7))
sns.barplot(x="feature",
            y="value",
            data=df.head(20),
           palette=df.head(20)["colors"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=20)
ax.set_title("Top 20 Features", fontsize=25)
ax.set_ylabel("Coef", fontsize=22)
ax.set_xlabel("Feature Name", fontsize=22)